In [1]:
import z3

Assume we have the following circuit:
```
A    B 
 \  /
 AND0    C
 /  \0  /
     OR0
      |1
     out
```
Note that `AND`s have 2 outputs, though only one is used. ORs have only 1.
On each wire there is an unshown "translator gate" (labeled with an integer on the wire), translating from the output of one gate to the input of the next. This is not the case for the wires between inputs and gates, however.

It may be the case that there are actually translators between inputs and gates---I need to ask Chris.


Each gate gets the following variables:

1. A variable for each of the inputs and outputs, representing the input's/output's species, and
2. A variable for the gate itself, indicating the gate's ID



In [2]:
and0_input0 = z3.Int("and0_input0")
and0_input1 = z3.Int("and0_input1")
and0_output0 = z3.Int("and0_output0")
and0_output1 = z3.Int("and0_output1")
and0 = z3.Int("and0")

gates = [(and0, (and0_input0, and0_input1, and0_output0, and0_output1))]

translator0_input = z3.Int("translator0_input")
translator0_output = z3.Int("translator0_output")
translator0 = z3.Int("translator0")

or0_input0 = z3.Int("or0_input0")
or0_input1 = z3.Int("or0_input1")
or0_output = z3.Int("or0_output")

translator1_input = z3.Int("translator1_input")
translator1_output = z3.Int("translator1_output")
translator1 = z3.Int("translator1")

translators = [
    (translator0, (translator0_input, translator0_output)),
    (translator1, (translator1_input, translator1_output)),
]

Note that `OR` gates don't actually have a gate associated with them! `OR` gates are not gates---they occur "naturally", when multiple translators map to the same species!

We wire up the circuit shown above by declaring constraints about which species must match:

In [3]:
a = z3.IntVal(100)
b = z3.IntVal(101)
c = z3.IntVal(102)
out = z3.IntVal(103)

s = z3.Solver()
s.add(and0_input0 == a)
s.add(and0_input1 == b)
s.add(translator0_input == and0_output0)
s.add(or0_input0 == translator0_output)
s.add(or0_input1 == c)
s.add(or0_output == or0_input0)
s.add(or0_output == or0_input1)
s.add(translator1_input == or0_output)
s.add(out == translator1_output)

Now we set some other constraints. We start with gate and translator constraints. Each gate or translator must be assigned to a valid gate or translator from a library of gates and a library of translators:

In [4]:
gate_library = [
    # Each gate stores its id, and the species ids of its inputs and outputs:
    # (gate_id, (in0, in1, out0, out1))
    (0, (0, 1, 2, 3))
]

for (gate, (i0, i1, o0, o1)) in gates:
    for (lgate, (li0, li1, lo0, lo1)) in gate_library:
        s.add(z3.Implies(gate == lgate, i0 == li0))
        s.add(z3.Implies(gate == lgate, i1 == li1))
        s.add(z3.Implies(gate == lgate, o0 == lo0))
        s.add(z3.Implies(gate == lgate, o1 == lo1))

Also, no gate can be used twice:
(I'm not sure how correct this is!)

In [5]:
for (gate, _) in gates:
    for (other_gate, _) in gates:
        if gate == other_gate:
            continue
        s.add(gate != other_gate)

Each gate must be assigned to a valid gate in the library:

In [6]:
for (gate, _) in gates:
    or_expr = False
    for (lgate, _) in gate_library:
        or_expr = z3.Or(or_expr, (lgate == gate))
    s.add(or_expr)

In [7]:
translator_library = [
    # Each translator stores its id, and the species ids of its input and output:
    # (translator_id, (in, out))
    (0, (0, 2))
]

for (translator, (i, o)) in translators:
    for (ltranslator, (li, lo)) in translator_library:
        s.add(z3.Implies(translator == ltranslator, i == li))
        s.add(z3.Implies(translator == ltranslator, o == lo))

In [8]:
for (translator, _) in translators:
    for (other_translator, _) in translators:
        if translator == other_translator:
            continue
        s.add(translator != other_translator)

In [9]:
for (translator, _) in translators:
    or_expr = False
    for (ltranslator, _) in translator_library:
        or_expr = z3.Or(or_expr, (ltranslator == translator))
    s.add(or_expr)

In [10]:
print(s)

[100 == and0_input0,
 101 == and0_input1,
 translator0_input == and0_output0,
 or0_input0 == translator0_output,
 102 == or0_input1,
 or0_output == or0_input0,
 or0_output == or0_input1,
 translator1_input == or0_output,
 103 == translator1_output,
 Implies(and0 == 0, and0_input0 == 0),
 Implies(and0 == 0, and0_input1 == 1),
 Implies(and0 == 0, and0_output0 == 2),
 Implies(and0 == 0, and0_output1 == 3),
 Or(False, and0 == 0),
 Implies(translator0 == 0, translator0_input == 0),
 Implies(translator0 == 0, translator0_output == 2),
 Implies(translator1 == 0, translator1_input == 0),
 Implies(translator1 == 0, translator1_output == 2),
 translator0 != translator1,
 translator1 != translator0,
 Or(False, translator0 == 0),
 Or(False, translator1 == 0)]


In [11]:
print(s.check())
if s.check() != z3.unsat:
    print(s.model())

unsat
